In [1]:
import pandas as pd
import numpy as np

In [21]:
# Create the frames
Highschooler = pd.DataFrame({
        'ID':[1510,1689,1381,1709, 1101, 1782, 1468,
              1641,1247, 1316,1911, 1501,1304,1025,
              1934,1661],
        'name':['Jordan', 'Gabriel', 'Tiffany', 
                'Cassandra','Haley', 'Andrew',
                'Kris', 'Brittany', 'Alexis',
                'Austin', 'Gabriel', 'Jessica',
                'Jordan', 'John', 'Kyle', 'Logan'],
        'grade':[9, 9, 9, 9, 10, 10, 10, 10, 11,
                 11, 11, 11, 12, 12, 12, 12]
    })
Friend = pd.DataFrame({
        'ID1':[1510, 1510, 1689, 1381, 1709, 1689,
               1782, 1782, 1782, 1468, 1468, 1101,
               1247, 1247, 1911, 1501, 1316, 1934,
               1304, 1661],
        'ID2':[1381, 1689, 1709, 1247, 1247, 1782,
               1468, 1316, 1304, 1101, 1641, 1641,
               1911, 1501, 1501, 1934, 1934, 1304,
               1661, 1025]
    })
Likes = pd.DataFrame({
        'ID1':[1689, 1709, 1782, 1911, 1247, 1641, 
               1316, 1501, 1934, 1025],
        'ID2':[1709, 1689, 1709, 1247, 1468, 1468, 
               1304, 1934, 1501, 1101]
    })